### USING TENSORBOARD FOR VISUALIZATION

+ TensorBoard is a great Interactive Visualization that can use for viewing the
    + learning curve during training
    + compare learning curves between multiple runs
    + visulize the computation graph
    + analyze training statistics
    + view images generated by the model
    + visualize complex multidimensional data projected down to 3D

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [10]:
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'.\\my_logs\\run_2023_02_02-11_44_04'

In [11]:
## use keras to load the dataset
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print("===================Shape of the Full Training and Test Set======================")
print(X_train_full.shape)
print(y_train_full.shape)
print(X_test.shape)
print(y_test.shape)

## the dataset is already splitted to training and test set.
## now the data is to be splitted to validation set.
X_valid, X_train = X_train_full[:5000]/255.0, X_train_full[5000:]/255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
print("===================Shape of the Training and Test Set after the Splitting to Validation Set======================")
print(X_valid.shape)
print(X_train.shape)
print(y_valid.shape)
print(y_train.shape)

## for fashion mnist, the class names are
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]
print(class_names)

## first example in the trainning set 
print(y_train[2])
print(class_names[y_train[0]])
print(class_names[y_train[2]])

print("==============Building the Model using Sequntial API=======================")
##defining the model
## this is the line to reate the sequential model
## simplest kind of keras model for neural networks, this is called sequential API.
model = keras.models.Sequential()
## First Layer is Flatten Layer
## This will convert each input image to 1D array.
## here we have to specify the input shape  = (28,28)
## we dont need to give the batch size here. only the input shape.
## alternatively we can give keras.layers.InputLayer(input_shape=[28,28])
model.add(keras.layers.Flatten(input_shape = [28,28]))
## next layer is dense layer with 300 neurons.
## it will use ReLU activation function.
model.add(keras.layers.Dense(300, activation = "relu"))
## next layer is also a Dense layer with 100 neurons
## it also uses the ReLU activation function.
model.add(keras.layers.Dense(100, activation = "relu"))
## next layer is the final layer with 10 neurons. to classify the 10 classes.
## here using the softmax actiavtion function.
model.add(keras.layers.Dense(10, activation = "softmax"))

# ## we can do the same thing in the given below manner as well
# model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[28,28]),
#     keras.layers.Dense(300, activation = "relu"),
#     keras.layers.Dense(100, activation = "relu"),
#     keras.layers.Dense(10, activation = "softmax")
# ])

## printing the model summary
print(model.summary())
## print the model layers
print(model.layers)
## check the first hidden layer
hidden1 = model.layers[1]
print(hidden1.name)
## checking the weights and biases.
weights, biases = hidden1.get_weights()
print(weights)
print(weights.shape)
print(biases)
print(biases.shape)

print("=================Compiling the model=====================")
## compile the model
## compile is to specify the loss fnuction and optimizer to use while training and evaluating.
## also we can give the metrics to use while training and evaluating model.
## using loss = "sparse_categorical_crossentropy"
## using optimizer as "sgd" which is stochcastic gradient descent.
## using the metrics as "accuracy"
model.compile(loss = "sparse_categorical_crossentropy",
                 optimizer = "sgd",
                 metrics = ["accuracy"])

print("================Use Callbacks============================")
checkpoint_cb = keras.callbacks.ModelCheckpoint("fashion_mnist_classifier_model_earlystopping_and_tesnorboard.h5", save_best_only=True)
print("================USING EARLY STOPPING METHODS===============")
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
print("================Training the Model=======================")
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs = 100, 
                    validation_data = [X_valid, y_valid],
                    callbacks=[checkpoint_cb,early_stopping_cb, tensorboard_cb])

===================Shape of the Full Training and Test Set======================
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
===================Shape of the Training and Test Set after the Splitting to Validation Set======================
(5000, 28, 28)
(55000, 28, 28)
(5000,)
(55000,)
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
7
Coat
Sneaker
==============Building the Model using Sequntial API=======================
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                           

In [18]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 12252), started 0:01:15 ago. (Use '!kill 12252' to kill it.)